In [1]:
import os
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn\\research'

In [3]:
## command should be in CustomerChurn directory
os.chdir('../')
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn'

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path


In [21]:
from ChurnPrediction.constants import *
from ChurnPrediction.utils.common import read_yaml, create_directory

In [22]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file
        )

        return data_ingestion_config
    


In [23]:
import os
import urllib.request as request
from ChurnPrediction import logger
from zipapp import zipfile
from src.ChurnPrediction.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
            Downloads the data from the url specified in the config
            return: None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'Downloaded {filename} to {self.config.local_data_file} with following info {headers}')

        else:
            logger.info(f'File {self.config.local_data_file} already exists')

    def extraact_zip_file(self):
        """
            Unzips the downloaded file
            return: None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    


In [1]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

NameError: name 'ConfigurationManager' is not defined